In [37]:
%pip install -qU langchain-text-splitters
%pip install qdrant-client
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [38]:
from pypdf import PdfReader
import os
FILE_PATH = os.path.join("data",r"C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf")
reader = PdfReader(FILE_PATH)
number_of_pages = len(reader.pages)

entire_text = ""
for page_num in range(number_of_pages):
    page = reader.pages[page_num]
    entire_text += page.extract_text()

entire_text[:200]

'SECTION II\nDiagnostic Criteria and Codes\nNeurodevelopmental Disorders\nSchizophrenia Spectrum and Other Psychotic Disorders\nBipolar and Related Disorders\nDepressive Disorders\nAnxiety Disorders\nObsessiv'

In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [40]:
text_chunks = text_splitter.split_text(entire_text)
print(f"Total chunks: {len(text_chunks)}")

Total chunks: 3538


In [41]:
text_chunks[:]

['SECTION II\nDiagnostic Criteria and Codes\nNeurodevelopmental Disorders\nSchizophrenia Spectrum and Other Psychotic Disorders\nBipolar and Related Disorders\nDepressive Disorders\nAnxiety Disorders\nObsessive-Compulsive and Related Disorders\nTrauma- and Stressor-Related Disorders\nDissociative Disorders\nSomatic Symptom and Related Disorders\nFeeding and Eating Disorders\nElimination Disorders\nSleep-Wake Disorders\nSexual Dysfunctions\nGender Dysphoria\nDisruptive, Impulse-Control, and Conduct Disorders\nSubstance-Related and Addictive Disorders\nNeurocognitive Disorders\nPersonality Disorders\nParaphilic Disorders\nOther Mental Disorders and Additional Codes\nMedication-Induced Movement Disorders and Other Adverse Effects of Medication\nOther Conditions That May Be a Focus of Clinical Attention\n3233This section\n contains the diagnostic criteria approved for routine clinical use along\nwith ICD-10-CM diagnostic codes. For each mental disorder, the diagnostic criteria are',
 'with

In [42]:
from llama_index.core.node_parser import SentenceSplitter

llamaindex_splitter = SentenceSplitter(chunk_size=1000, chunk_overlap=50)
llamaindex_text_chunks = llamaindex_splitter.split_text(entire_text)

In [43]:
!pip install sentence-transformers

import torch
from sentence_transformers import SentenceTransformer

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model_name = "BAAI/bge-small-en-v1.5"
model_name = "all-MiniLM-L6-v2"

embedding_model = SentenceTransformer(model_name, device=device)

In [44]:
embeddings = embedding_model.encode(text_chunks, show_progress_bar=True)

Batches:   0%|          | 0/111 [00:00<?, ?it/s]

In [45]:
 embeddings[0].shape

(384,)

In [46]:
!pip install faiss-cpu sentence-transformers


In [47]:
import faiss
import numpy as np

# Convert embeddings to float32 (FAISS requirement)
embeddings = np.array(embeddings).astype("float32")

# Normalize for cosine similarity
faiss.normalize_L2(embeddings)

# Get dimension of embeddings
dimension = embeddings.shape[1]

# Create FAISS index for cosine similarity
index = faiss.IndexFlatIP(dimension)

# Add vectors to index
index.add(embeddings)

print(f"✅ FAISS index created with {index.ntotal} vectors")



✅ FAISS index created with 3538 vectors


In [48]:
payloads = [{"source": FILE_PATH, "content": chunk} for chunk in text_chunks]

In [49]:
# Search query
query = "what is Bipolar and Related Disorders?"
query_vector = embedding_model.encode([query]).astype("float32")
faiss.normalize_L2(query_vector)

# Search top 3 results
distances, indices = index.search(query_vector, k=3)

# Display matches
for idx, score in zip(indices[0], distances[0]):
    print(f"\nScore: {score:.4f}")
    print(f"Source: {payloads[idx]['source']}")
    print(f"Content: {payloads[idx]['content'][:200]}...")



Score: 0.6962
Source: C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf
Content: depressive disorder. About a third of individuals with bipolar II disorder reported a family
history of bipolar disorder. There may be genetic factors influencing the age at onset for bipolar
disorder...

Score: 0.6833
Source: C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf
Content: history, and genetics. The diagnoses included in this chapter are bipolar I disorder, bipolar II
disorder, cyclothymic disorder, substance/medication-induced bipolar and related disorder,
bipolar and ...

Score: 0.6763
Source: C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf
Content: 146
may accompany the manic episode. Mood may shift very rapidly to anger or depression; some
individuals may become hostile and physically threatening to others and, when delusional,
become physicall...
